# `nbconvert_html5` interfaces

this project exports tools for making jupyter notebook products more accessible. currently, this project features:
* an `nbconvert` template with that uses html5 semantics.

In [1]:
    from nbconvert import  get_exporter, get_export_names
    import nbconvert_html5, pathlib

this module exports the `html5` nbconvert template.

In [2]:
    assert "html5" in get_export_names(), "the entry point is not defined."

the two templates - `html` and `html5` - can be loaded and compared.

In [3]:
    html, html5 = get_exporter("html"), get_exporter("html5")

## remediations

the major changes to `html5` version are that sections and articles have been added to the cells.

In [4]:
    A = html().from_filename("notebooks/lorenz-executed.ipynb")[0]
    B = html5().from_filename("notebooks/lorenz-executed.ipynb")[0]
    A, B = map(nbconvert_html5.modifiers.Soup, (A, B))

* adding a `main` landmark

In [5]:
    A.select("section"), len(B.select("main"))

([], 1)

* cells are articles

In [6]:
    A.select("article"), len(B.select("article.jp-Cell"))

([], 16)

* outputs are sections

In [7]:
    A.select("section"), len(B.select("section.jp-OutputArea"))

([], 5)

## using the configuration to modify tests

`jupyter`s configuration system allows us to modify html representations on the fly.
see the `jupyter_nbconvert_config.py` for an example where we convert prompts to asides.

In [8]:
    cli = !jupyter nbconvert --to html5 --config jupyter_nbconvert_config.py notebooks/lorenz-executed.ipynb --stdout
    target = "".join(cli)
    assert "<aside" in target, target